In [1]:
import os
import numpy as np # fast array library
import pandas as pd # data frames library
# plot display type. use this to keep graphs within the notebook
import matplotlib 

import matplotlib.pyplot as plt # plot library
   
import numpy.ma as ma

import scipy.constants as ct # import mathematical constants
from scipy import stats # statistics, for binning and fields calculation

import scipy.interpolate as interpol

from matplotlib import rc   
rc('text', usetex=True)
plt.rcParams['font.size'] = 14

import ipywidgets as widgets # import 

# Pandas table columns to 2D arrays (matrices)
# necessary for 2D plots
# df: input pandas dataframe;
# nx = number of bins in X axis; idem for ny
def fields_to_matrix(df, nx, ny):
    global X, Y, ux, uy, T, n, speed
    Y = np.zeros((15,15))
    X = np.zeros((15,15))
    ux = np.zeros((15,15))
    uy = np.zeros((15,15))
    T = np.zeros((15,15))
    n = np.zeros((15,15)) 
    for i in range(ny):
        for j in range(nx):
            Y[i][j] = df['Y'][nx*i + j]
            X[i][j] = df['X'][nx*i + j]
            ux[i][j] = df['ux'][nx*i + j]
            uy[i][j] = df['uy'][nx*i + j]
            n[i][j] = df['n'][nx*i + j]
            T[i][j] = df['T'][nx*i +j]
    speed = np.sqrt(ux**2 + uy**2)


In [5]:
plt.style.available

['seaborn-deep',
 'seaborn-paper',
 'seaborn-white',
 'seaborn-bright',
 'seaborn-whitegrid',
 'Solarize_Light2',
 'fivethirtyeight',
 'seaborn-notebook',
 'seaborn-darkgrid',
 'dark_background',
 'fast',
 'seaborn-poster',
 'seaborn-ticks',
 'classic',
 'ggplot',
 'bmh',
 'seaborn-talk',
 '_classic_test',
 'seaborn-dark',
 'seaborn-colorblind',
 'tableau-colorblind10',
 'seaborn-dark-palette',
 'seaborn-muted',
 'seaborn-pastel',
 'seaborn',
 'grayscale']

In [2]:
%matplotlib tk

In [7]:
direc = '/data/Datos/blades/fields/N55/'
file = 'fields_N55_T_396b48aaefee060ae012b6d80425b879.dat'

name = direc + file
datos = pd.read_csv(name,sep='\s+')

fields_to_matrix(datos,15,15) 

In [8]:
figure, axes = plt.subplots(figsize=(7,7))
plt.xlim((-400,400))
plt.ylim((-400,400))

draw_circle = plt.Circle((0, 0), 400, fill=False, color='y', linewidth=4)
axes.set_aspect(1)
axes.add_artist(draw_circle)

W = np.diff(uy,axis=1)[:-1,:] - np.diff(ux,axis=0)[:,:-1]
plt.xlim((-400,400))
plt.ylim((-400,400))
plt.contourf(X[:-1,:-1] + 400/15,Y[:-1,:-1]+400/15, W, cmap='seismic', vmin = -0.5*np.max(np.abs(W)), vmax=0.5*np.max(np.abs(W)), levels=np.linspace(-1,1,40))
plt.colorbar(fraction=0.047, pad=0.01)
plt.streamplot(X, Y, ux, uy, color='k', density=2.6, linewidth=1)
draw_circle = plt.Circle((0, 0), 400, fill=False, color='y', linewidth=30)

In [9]:
x = np.linspace(-400, 400, 15)
y = np.linspace(-400, 400, 15)

interp_spline_x = interpol.RectBivariateSpline(x, y, ux)
interp_spline_y = interpol.RectBivariateSpline(x, y, uy)


x2 = np.linspace(-400, 400, 400)
y2 = np.linspace(-400, 400, 400)
X2, Y2 = np.meshgrid(x2,y2)
ux2 = interp_spline_x(y2, x2)
uy2 = interp_spline_y(y2, x2)
speed2 = np.sqrt(ux2**2 + uy2**2)

# axis 0 is difference along the Y axis; axis 1 (default) is X axis
# see 
W2 =  np.diff(uy2,axis=1)[:-1,:] - np.diff(ux2,axis=0)[:,:-1]

figure, axes = plt.subplots(figsize=(7,7))
plt.xlim(-420,420)
plt.ylim(-420,420)


ax = plt.axes() 
ax.set_facecolor('whitesmoke') 

draw_circle = plt.Circle((0, 0), 405, fill=False, color='y', linewidth=4)
ax.set_aspect(1)
ax.add_artist(draw_circle)

mW2 =  ma.masked_where(X2[:-1,:-1]**2+Y2[:-1,:-1]**2>405**2, W2/np.max(np.abs(W2)))
mux2 = ma.masked_where(X2**2+Y2**2>390**2, ux2)
muy2 = ma.masked_where(X2**2+Y2**2>390**2, uy2)

plt.contourf(X2[:-1,:-1],Y2[:-1,:-1], mW2, cmap='seismic_r',  levels=np.linspace(-0.85,0.85,20))
plt.colorbar(fraction=0.047, pad=0.01)

lw = 5*speed2 / speed2.max()
st = plt.streamplot(X2,Y2,mux2,muy2, density=1.75, color='k',linewidth=lw)  
st.lines.set_alpha(0.5)
#for arrow_patch in plt.gca().patches[-len(st.arrows.get_paths()):]:
#    arrow_patch.set_alpha(0.5)
#plt.savefig('fig2b2.pdf')

In [62]:
plt.close('all')

In [42]:
Y2[0,0]

-400.0

In [40]:
np.min(Y2)

-383.0

In [6]:
plt.figure(figsize=(7,7))
plt.streamplot(X2,Y2,ux2,uy2, density=2.0, color='b')
plt.streamplot(X,Y,ux,uy, density=2.0, color='k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(15, 15),\
    gridspec_kw={'width_ratios': [3, 3],'height_ratios': [2.7, 3], 'wspace': 0.31, 'hspace':0.15})


#### FLOW FIELD (using streamplot) ####
draw_circle = plt.Circle((0, 0), 408, fill=False, color='b', linewidth=3)
ax1.set_aspect(1)
ax1.add_artist(draw_circle)
lw = 10 * speed
plt.xlim(-420,420)
plt.ylim(-420,420)
stream_container = ax1.streamplot(X, Y, ux, uy, color='xkcd:azure', density=2, linewidth=lw)

plt.xlim(-400,400)
plt.ylim(-400,400)

#### TEMPERATURE FIELD ##############
temp = ax2.imshow(T,cmap='rainbow', interpolation='gaussian',extent=(-400,400,-400,400),  )
cbar2 = plt.colorbar(temp, ax=ax2, fraction=0.047, pad=0.01)
cbar2.set_label(r'$T/m$', rotation=270, labelpad=25, fontsize=18)

####  PRESSURE FIELD ##############
press = ax3.imshow(n*T, interpolation='gaussian', extent=(-400,400,-400,400), \
                     vmax = 0.75 * np.max(n*T), vmin = 5* np.min((n*T)[n*T>0]) )
cbar3 = plt.colorbar(press, ax=ax3, fraction=0.047, pad=0.01)
cbar3.set_label(r'$P$', rotation=270, labelpad=25, fontsize=18)

####  DENSITY FIELD ##############

dens = ax4.imshow(n, interpolation='gaussian', cmap='gnuplot', extent=(-400,400,-400,400), \
                 vmax = 0.75 * np.max(n), vmin = 5*np.min(n[n>0]))
cbar4 = plt.colorbar(dens, ax=ax4,fraction=0.047, pad=0.01)
cbar4.set_label(r'$n/\bar{n}$', rotation=270, labelpad=25, fontsize=18)

/usr/lib/python3/dist-packages/matplotlib/__init__.py:882: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## $T/m = 0.40$

In [77]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(15, 15),\
    gridspec_kw={'width_ratios': [3, 3],'height_ratios': [2.7, 3], 'wspace': 0.31, 'hspace':0.15})


#### FLOW FIELD (using streamplot) ####
draw_circle = plt.Circle((0, 0), 408, fill=False, color='b', linewidth=3)
ax1.set_aspect(1)
ax1.add_artist(draw_circle)
lw = 10 * speed
plt.xlim(-420,420)
plt.ylim(-420,420)
stream_container = ax1.streamplot(X, Y, ux, uy, color='xkcd:azure', density=2.6, linewidth=lw)

plt.xlim(-400,400)
plt.ylim(-400,400)

#### TEMPERATURE FIELD ##############
temp = ax2.imshow(T,cmap='rainbow', interpolation='gaussian',extent=(-400,400,-400,400), \
                  vmax = np.max(T)*0.75, vmin= 5*np.min(T[T>0]) )
cbar2 = plt.colorbar(temp, ax=ax2, fraction=0.047, pad=0.01)
cbar2.set_label(r'$T/m$', rotation=270, labelpad=25, fontsize=18)

####  PRESSURE FIELD ##############
press = ax3.imshow(n*T, interpolation='gaussian', extent=(-400,400,-400,400), \
                     vmax = 0.75 * np.max(n*T), vmin = 5* np.min((n*T)[n*T>0]) )
cbar3 = plt.colorbar(press, ax=ax3, fraction=0.047, pad=0.01)
cbar3.set_label(r'$P$', rotation=270, labelpad=25, fontsize=18)

####  DENSITY FIELD ##############

dens = ax4.imshow(n, interpolation='gaussian', cmap='gnuplot', extent=(-400,400,-400,400), \
                 vmax = 0.75 * np.max(n), vmin = 5*np.min(n[n>0]))
cbar4 = plt.colorbar(dens, ax=ax4,fraction=0.047, pad=0.01)
cbar4.set_label(r'$n/\bar{n}$', rotation=270, labelpad=25, fontsize=18)


In [166]:
def read_momentsRr(npart):
    direc = '/data/Datos/blades/fields/bin3/'
    name = direc + 'moments_N' + str(npart) + '.dat'
    momentsR = pd.read_csv(name, sep='\s+')
    return momentsR
    

In [167]:
fr0.index.values

array([0, 1, 2, 3, 4])

In [172]:
momentsR = read_momentsRr(25)
fr0 = momentsR[momentsR.r == momentsR.r[0]].sort_values(by=['Tt'],ignore_index=True )
fr1 = momentsR[momentsR.r == momentsR.r[1]].sort_values(by=['Tt'],ignore_index=True )
fr2 = momentsR[momentsR.r == momentsR.r[2]].sort_values(by=['Tt'],ignore_index=True )

plt.figure()

plt.plot(fr0.index.values,(fr0.a11+1)*fr0.Tt/fr0.Tr-1, 'o', color='r')
plt.plot(fr0.index.values,(fr1.a11+1)*fr1.Tt/fr1.Tr-1, 'o', color='g')
plt.plot(fr0.index.values,(fr2.a11+1)*fr2.Tt/fr2.Tr-1, 'o', color='b')

plt.figure()

plt.plot(fr0.index.values,fr0.Tt/fr0.Tr, 'o', color='r')
plt.plot(fr0.index.values,fr1.Tt/fr1.Tr, 'o', color='g')
plt.plot(fr0.index.values,fr2.Tt/fr2.Tr, 'o', color='b')

plt.figure()

plt.plot(fr0.index.values,fr0.Tr, 'o', color='r')
plt.plot(fr0.index.values,fr1.Tr, 'o', color='g')
plt.plot(fr0.index.values,fr2.Tr, 'o', color='b')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/usr/lib/python3/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/usr/lib/python3/dist-packages/matplotlib/__init__.py:882: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/usr/lib/python3/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/usr/lib/python3/dist-packages/matplotlib/__init__.py:882: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [141]:
fr0.n, fr1.n, fr2.n

(0     230888
 12    244327
 9     157535
 6     145252
 3     147426
 Name: n, dtype: int64,
 1     624834
 13    545960
 10    448234
 7     439623
 4     445975
 Name: n, dtype: int64,
 2     231418
 14    220503
 11    399726
 8     360053
 5     412928
 Name: n, dtype: int64)